# anchor_generator
**This file used to generate anchors for UECFOOD256 dataset through kmeans.
Distance here is not Euclidean distance but IoU (Intersection over Union) in
accordance with original yolo-v2 paper.**

In [1]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np

**k-means clustering**:  
- Input: k, set of points [w1,h1], [w2,h2], [w3, h3], ..., [wn, hn] 
- Place centroids c1, ..., ck at random locations (randomly select k [w,h] among input img wh)
- Repeat until convergence:
    - for each point [wi,hi]:
        - find nearest centroid cj. (np.argmin(new_distances, axis=1))
        - assign the point [wi,hi] to cluster j.
    - for each cluster j=1, ...,k: (update centroids)
        - new centroid cj=mean of all points [wi,hi] assigned to cluster j in previous step
        $\frac{1}{n_j}\sum\limits_{x_i\rightarrow c_j}x_i(a)$  (a means a particular attribute in this case a is wh[i])
- Stop when none of the cluster assignments change

In [2]:
def kmeans(wh, centroids, anchor_txt):
    num = wh.shape[0]  # total number of different wh pairs
    k, dim = centroids.shape
    iter = 0
    old_distances = np.zeros((num, k))
    _assignments = -np.ones(num)

    # iterate until
    while True:
        new_distances = []
        iter += 1
        for i in range(num):
            distance = 1 - IoU(wh[i], centroids)  # high IoU represents low distance
            new_distances.append(distance)
        new_distances = np.array(new_distances)
        print('Iter {}: distances: {}'.format(iter, np.sum((np.abs(old_distances - new_distances)))))

        # for each input img assign a centroid (select the closed one)
        assignments = np.argmin(new_distances, axis=1)
        if (assignments == _assignments).all():
            print('final centroids =', centroids)
            save_anchors(centroids, anchor_txt, wh_in_yolov2)
            return centroids
        else:
            centroid_sums = np.zeros((k, dim), np.float)
            for i in range(num):
                centroid_sums[assignments[i]] += wh[i]  # sum up attribute
            for j in range(k):
                # new centroids
                centroids[j] = centroid_sums[j] / np.sum(assignments == j)

            _assignments = assignments.copy()
            old_distances = new_distances.copy()

In [3]:
def save_anchors(centroids, anchor_txt, wh_in_yolov2):
    width_in_yolov2 = wh_in_yolov2[0]
    height_in_yolov2 = wh_in_yolov2[1]
    with open(anchor_txt, 'w') as file:
        anchors = centroids.copy()
        for i in range(anchors.shape[0]):
            anchors[i][0] *= width_in_yolov2 / 32.
            anchors[i][1] *= height_in_yolov2 / 32.
        widths = anchors[:, 0]
        sorted_indices = np.argsort(widths)  # return the indices that sort tht array
        print('anchors = ', anchors[sorted_indices])

        for i in sorted_indices:
            file.write('%0.2f, %0.2f\n' % (anchors[i, 0], anchors[i, 1]))

In [4]:
def avgIoU(wh, centroids):
    sum = 0.
    for i in range(wh.shape[0]):
        sum += max(IoU(wh[i], centroids))
    return sum / wh.shape[0]

**IoU - Intersection over Union **

In [5]:
def IoU(whi, centroids):
    """ Calculate IoU between current centroids with one in wh array to check if current
    centroids are suitable enough
    :param whi:
    :param centroids:
    :return:
    """
    IOU = []
    for centroid in centroids:
        c_w, c_h = centroid
        w, h = whi
        if c_w >= w and c_h >= h:
            iou = w * h / (c_w * c_h)
        elif c_w >= w and c_h <= h:
            iou = w * c_h / (w * h + (c_w - w) * c_h)
        elif c_w <= w and c_h >= h:
            iou = c_w * h / (w * h + (c_h - h) * c_w)
        else:
            iou = c_w * c_h / (w * h)
        IOU.append(iou)
    return np.array(IOU)

In [6]:
def coordinate2wh(coordinates, uec256_dims):
    coordinates = list(map(float, coordinates))
    w = (coordinates[2] - coordinates[0]) / uec256_dims[0]  # x2-x1
    h = (coordinates[3] - coordinates[1]) / uec256_dims[0]  # y2-y1
    return w, h

In [9]:
def gen_anchors(n_clusters, uec256_dims):
    dataset_disk = '/Volumes/JS/UECFOOD256/'
    output_path = dataset_disk + 'generated_anchors'
    train_uec256 = dataset_disk + 'UECFOOD256/train_uec256.txt'

    if not os.path.exists(output_path):
        os.mkdir(output_path)

    wh = []

    with open(train_uec256, 'r') as file:
        for i, line in enumerate(file):
            if i > 0:
                line = line.rstrip('\n')
                line = line.split(' ')
                coordinates = line[2:]
                w, h = coordinate2wh(coordinates, uec256_dims)
                wh.append([w, h])
        wh = np.array(wh)

        if n_clusters == 0:  # make from 1 to 10 clusters and pick the best one
            avgIou = []
            for n_cluster in range(1, 11):
                anchor_txt = os.path.join(output_path, 'anchors_%d.txt' % (n_cluster))
                # randomly select n_cluster anchors from wh array which contain w,h for each img
                indices = [random.randrange(wh.shape[0]) for i in range(n_cluster)]
                centroids = wh[indices]
                centroids = kmeans(wh, centroids, anchor_txt)
                avgIou.append([n_cluster, avgIoU(wh, centroids)])
            avgIou = np.array(avgIou)
            plt.plot(avgIou[:, 0], avgIou[:, 1])
            plt.scatter(avgIou[:, 0], avgIou[:, 1], c='r')
            plt.xlabel('number of cluster')
            plt.ylabel('average IoU')
            plt.savefig('avg_iou')
            plt.show()
        else:
            anchor_txt = os.path.join(output_path, 'anchors_%d.txt' % (n_clusters))
            # randomly select n_cluster anchors from wh array which contain w,h for each img
            indices = [random.randrange(wh.shape[0]) for i in range(n_clusters)]
            centroids = wh[indices]
            kmeans(wh, centroids, anchor_txt)

        print('Done!')

In [10]:
wh_in_yolov2 = [416, 416]
uec256_dims = [800, 600]  # dataset image width=800, height=600
n_clusters = 0

In [ ]:
gen_anchors(n_clusters, uec256_dims)

Iter 1: distances: 8476.403771152614
Iter 2: distances: 3290.723950043347
final centroids = [[0.50163718 0.40356145]]
anchors =  [[6.52128339 5.24629884]]
Iter 1: distances: 28258.378764419573
Iter 2: distances: 8610.7504373644
Iter 3: distances: 2339.1659692091016
Iter 4: distances: 1289.4360253382677
Iter 5: distances: 829.9807099064368
Iter 6: distances: 402.0808595680844
Iter 7: distances: 220.54991923393584
Iter 8: distances: 129.70145736604363
Iter 9: distances: 67.81279792640058
Iter 10: distances: 34.54363960392638
Iter 11: distances: 15.195475906479555
Iter 12: distances: 6.5698766046799575
final centroids = [[0.58453047 0.47443103]
 [0.30127868 0.23226502]]
anchors =  [[3.91662279 3.01944525]
 [7.59889612 6.16760345]]
Iter 1: distances: 31752.21538693832
Iter 2: distances: 4435.1142000980335
Iter 3: distances: 1184.9811127252665
Iter 4: distances: 766.1757708552572
Iter 5: distances: 710.3949646447923
Iter 6: distances: 549.7529664527419
Iter 7: distances: 410.322195660434
It

Iter 54: distances: 59.02937267931962
Iter 55: distances: 63.545913661593026
Iter 56: distances: 44.33380187662945
Iter 57: distances: 39.951950706814316
Iter 58: distances: 40.6080855519892
Iter 59: distances: 42.23217981567718
Iter 60: distances: 41.65338889081353
Iter 61: distances: 43.79444036147837
Iter 62: distances: 52.670258457688305
Iter 63: distances: 53.342760608522454
Iter 64: distances: 59.12452037277307
Iter 65: distances: 68.60290848695425
Iter 66: distances: 56.69484086797058
Iter 67: distances: 68.4672095890873
Iter 68: distances: 64.50198073929593
Iter 69: distances: 80.72462248370721
Iter 70: distances: 90.25300135838991
Iter 71: distances: 109.41571992868155
Iter 72: distances: 123.526769487786
Iter 73: distances: 132.94278211518724
Iter 74: distances: 147.8512553377758
Iter 75: distances: 185.81430732358888
Iter 76: distances: 215.77443757488172
Iter 77: distances: 330.018967849933
Iter 78: distances: 426.02532735330556
Iter 79: distances: 544.6909785379319
Iter 80

Iter 52: distances: 203.1341880405376
Iter 53: distances: 244.60608780521108
Iter 54: distances: 231.03287554533054
Iter 55: distances: 236.28759038254668
Iter 56: distances: 256.5405967786952
Iter 57: distances: 266.34686460046026
Iter 58: distances: 269.99878806476306
Iter 59: distances: 285.6746949791728
Iter 60: distances: 406.51736299561384
Iter 61: distances: 287.0186240044268
Iter 62: distances: 284.2580500106964
Iter 63: distances: 269.25177060247717
Iter 64: distances: 241.04668793726682
Iter 65: distances: 279.2848566173635
Iter 66: distances: 232.52729653428935
Iter 67: distances: 185.0350580019221
Iter 68: distances: 130.39447539015143
Iter 69: distances: 157.16995196422687
Iter 70: distances: 154.07582348274477
Iter 71: distances: 154.65237410300477
Iter 72: distances: 157.63182025048235
Iter 73: distances: 160.28165839430523
Iter 74: distances: 150.62002155627184
Iter 75: distances: 174.0478248232203
Iter 76: distances: 156.78848085694787
Iter 77: distances: 130.630979876